In [ ]:
from funcs import funcs
from Crosswalk.Transformer import Transformer
from Crosswalk.DataCache import DataCache
from Crosswalk.NDAWriter import NDAWriter
from Crosswalk.Manager import Manager
import Crosswalk.cleanupfuncs as cleanupfuncs
from Crosswalk.Loader import Loader, BoxLoader, BoxHcaLoader, QintLoader, QintHcaLoader, RedcapLoader
import os
import pandas as pd
import numpy as np
from ccf.easy_yaml import EasyYaml
from ccf.redcap import RedcapTable
print("hi")
print(os.getcwd())

In [ ]:
#create folder for prepped structures, if it doesn't exist
!!mkdir prepped
!!mkdir prepped/hca

from ccf.config import LoadSettings

config = LoadSettings()
config['rosetta']['filename']

In [ ]:
# note the path to the validator below.  This is the vtcmd.py validator written by the NDA.  
# If you haven't already installed this, please do so now: https://github.com/NDAR/nda-tools
# and the place the path that shows up when you type 'which vtcmd' from your terminal
# validation results will be sent to and read from whatever default is specified in the vtcmd configuration file,
# so if you're using vtcmd to validate any other datatypes, keep this in mind.


M = Manager(
        data =  DataCache(
            BoxHcaLoader('PennCNP',592325063896),
            RedcapLoader('hcpa'),          
            QintHcaLoader()
        ),#SsagaLoader(),
        writer = NDAWriter(completed_dir="/Users/petralenzini/work/Behavioral/Lifespan/NDA_submissions/NDA_submissions/prepped/hca/", validator="/opt/homebrew/bin/vtcmd"),
        #writer = NDAWriter(completed_dir="./prepped/hca/"),
        transformer = Transformer(funcs = funcs, map_dir='./maps/hca/')
)

#M = Manager(
#        data =  DataCache(RedcapLoader('hcpa')),
#        writer = NDAWriter(completed_dir="./prepped/hca/", validator="vtcmd"),
#        #writer = NDAWriter(completed_dir="./prepped/hca/"),
#        transformer = Transformer(funcs = funcs, map_dir='./maps/hca/'))

In [ ]:
# This step requires that you have a 'rosetta stone or inventory' file that has all the required NDA fields for 
# all subjects you intend to submit at this time.  This approach facilitates keeping track of subject counts
# across data types.  For example, if your required fields are already stored in XNAT because you had the CCF
# upload your imaging data for you, you can export this csv from XNAT and rename as appropriate.  
# Place this file at the main level of this repository, and name it in your config file
# Loader.py program's _post_load_hook_ method referenced below.  

#WHERE TO PUT THIS?
#inventory=pd.read_csv(extrainfo)
#inventory=inventory.loc[inventory.IntraDB != "CCF_PCMP_ITK"][['DB_Source','REDCap_id','subject','pseudo_guid','nda_interview_date','nda_age','M/F','pedid','redcap_event_name','redcap_event','race','ethnic_group','site']].copy()
###


M.preload_data(['PennCNP','qint','hcpa'])#,
#dir(M)

In [ ]:
#make sure scripts are appropriately configured:
M.run('psqi01')
cleanupfuncs.cleanlist(structurelist=['psqi01'],fPath="./prepped/hca/")

In [ ]:
#make sure scripts are appropriately configured:
M.run('trail_ca01')
cleanupfuncs.redcleanup(structure="trail_ca01",filePath="./prepped/hca/",extraomitcol1='seizures',extraomitcol2='versionchildadult')


In [ ]:
#make sure scripts are appropriately configured:
M.run('vitals01')
cleanupfuncs.cleanzeros()

In [ ]:
#SSAGA handled separately

In [ ]:
#note that vtcmd seems to fail if it gets too many requests in a row (?)
#seems to run fine if doing one by one on the command line and then coming back here to run.
#must be a secret cache thing somewhere

structs = [
    'lbadl01', #passed
    'mchq01', #passed
    'er4001', #passed 
    'deldisk01', #passed
    'asr01', #passed
    'batbil01',#passed
    'bsc01',#passed
    ## 'drugscr01', drugscr mapped incorrectly and revealed faulty collection logic.
    ## Taken out of 2.0 and 3.0 release until cleanup using paper backups. Substantial missingness likely.
    'gales01',#passed
    'ipaq01',#passed
    'leap01',#passed
    'medh01',#passed
    'mendt01',#passed
    'moca01',#passed
    'nffi01',
    'ravlt01',
    'scan_debrief01',
  ]

for s in structs:
    print(s)
    M.run(s)
    


In [ ]:
#rsptest_no
import pandas as pd
structure='bsc01'
filePath="./prepped/hca/"
strucroot = structure[:-2]
strucnum = structure[-2:]

In [ ]:
df=pd.read_csv(filePath + structure + ".csv", header=1)

In [ ]:

df.loc[df.rsptest_no>9000]
#with open(filePath + structure + ".csv", 'w') as f:
#        f.write(strucroot + "," + str(int(strucnum)) + "\n")
#        df.to_csv(f, index=False)
#df.rsptest_no

In [ ]:
cleanupfuncs.cleanlist(structurelist=structs,fPath="./prepped/hca/")

#cleanupfuncs.redcleanup(structure="bsc01", filePath="./prepped/hca/")
cleanupfuncs.asrover60(structure="asr01",filePath="./prepped/hca/")
cleanupfuncs.redcleanup(structure="asr01", filePath="./prepped/hca/", extraomitcol1='version_form', extraomitcol2='NO',
    extraomitcol3='NO', extraomitcol4='NO', extraomitcol5='NO')
cleanupfuncs.redcleanup(structure="deldisk01",filePath="./prepped/hca/",extraomitcol1='version_form',extraomitcol2='NO',
    extraomitcol3='NO', extraomitcol4='NO', extraomitcol5='NO')
cleanupfuncs.redcleanup(structure="medh01",filePath="./prepped/hca/",extraomitcol1='NO')
#cleanupfuncs.redcleanup(structure="bsc01",filePath="./prepped/hca/",extraomitcol1='NO')
cleanupfuncs.redcleanup(structure="ravlt01",filePath="./prepped/hca/",extraomitcol1='ravlt_delt',extraomitcol2='ravlt_disct',extraomitcol3='ravlt_tott')#

cleanupfuncs.redcleanup(structure="scan_debrief01",filePath="./prepped/hca/",extraomitcol1='NO')
cleanupfuncs.redcleanup(structure="er4001",filePath="./prepped/hca/",extraomitcol1='NO')
cleanupfuncs.redcleanup(structure="lbadl01", filePath="./prepped/hca/",extraomitcol1='NO')
cleanupfuncs.redcleanup(structure="mchq01", filePath="./prepped/hca/",extraomitcol1='NO')
cleanupfuncs.redcleanup(structure="nffi01", filePath="./prepped/hca/",extraomitcol1='neo2_score_ne',extraomitcol2='neo2_score_ex',extraomitcol3='neo2_score_op',extraomitcol4='neo2_score_ag',extraomitcol5='neo2_score_co')

In [ ]:
#fix noninteger types
a1=['asr1_1','asr1_3','asr1_5','asr4_3','asr4_4','asr4_5','asr4_6','asr5_1','asr5_2','asr5_4','asr6_6','asr7_1','asr7_3','asr7_6','asr8_1','asr8_4','asr8_5','asr11_6','asr12_2','asr12_4','asr12_6','asr13_3','asr13_4','asr14_2','asr14_3','asr14_6','asr15_2','asr17_2','asr19_2','asr19_6','asr20_3','asr21_3','asr22_1','asr22_2','asr22_3','asr22_5','asr3_1','asr6_4','oasr_ppl35','oasr_ppl3','oasr_ppl39','asr8_6','oasr_ppl16','oasr_ppl19','oasr_ppl20','oasr_ppl47','oasr_ppl49_10','oasr_ppl49_11','oasr_ppl54','oasr_ppl79','oasr_ppl101','oasr_ppl102','oasr_ppl109','oasr_ppl110','oasr_ppl114','oasr_ppl121','oasr_ppl33']
a2=['asr2_2','asr2_3','asr2_4','asr3_2','asr3_4','asr3_6','asr5_5','asr6_5','asr7_4','asr7_5','asr8_3','asr9_2','asr9_4','asr9_6','asr10_3','asr10_4','asr10_5','asr10_6','asr11_1','asr11_2','asr11_3','asr11_4','asr11_5','asr12_3','asr14_1','asr15_1','asr15_3','asr16_1','asr16_3','asr16_4','asr17_4','asr17_5','asr18_2','asr18_4','asr19_1','asr19_3','asr19_4','asr20_2','asr20_6','asr21_1','asr21_4']
a=a1+a2
cleanupfuncs.integercleanup(structure='asr01',filePath="./prepped/hca/",varlist=a)

In [ ]:
b1=['batlq22_lc','batlq22_dz','batlq32_lc','batlq32_dz','batlq15_lc','batlq15_dz','batlq4','batlq8','batlq_score1','batlq_score10','batlq_score11','batlq_score13','batlq_score14','batlq_score16','batlq_score17','batlq_score18','batlq_score19','batlq_score2','batlq_score20','batlq_score23','batlq_score24','batlq_score25','batlq_score26','batlq_score27','batlq_score28','batlq_score29','batlq_score3','batlq_score30','batlq_score31','batlq_score33','batlq_score39','batlq_score4','batlq_score5','batlq_score6','batlq_score7','batlq_score8','batlq_score9','batlq1_lc','batlq1_dz','batlq16_lc','batlq16_dz','batlq23_lc','batlq23_dz','batlq14_lc','batlq14_dz','batlq18_lc','batlq18_dz','batlq20_lc','batlq20_dz','batlq39_lc','batlq39_dz','batlq9_lc','batlq9_dz','batlq24_lc','batlq24_dz','batlq10_lc','batlq10_dz','batlq5_lc','batlq5_dz','batlq27_lc','batlq27_dz','batlq11_lc','batlq11_dz','batlq7_lc','batlq7_dz','batlq19_lc','batlq19_dz','batlq25_lc','batlq25_dz','batlq4_lc','batlq4_dz','batlq6_lc','batlq6_dz','batlq17_lc','batlq17_dz','batlq35_lc','batlq35_dz','batlq38_lc','batlq38_dz','batlq13_lc','batlq13_dz','batlq33_lc','batlq33_dz','batlq8_lc','batlq8_dz','batlq2_lc','batlq2_dz','batlq26_lc','batlq26_dz','batlq36_lc','batlq36_dz','batlq34_lc','batlq34_dz','batlq12_lc','batlq12_dz','batlq31_lc','batlq31_dz','batlq21_lc','batlq21_dz','batlq29_lc','batlq29_dz','batlq3_lc','batlq3_dz','batlq28_lc','batlq28_dz','batlq30_lc','batlq30_dz']
b2=['batlq1','batlq2','batlq3','batlq5','batlq6','batlq7','batlq9','batlq10','batlq11','batlq12','batlq13','batlq14','batlq15','batlq16','batlq17','batlq18','batlq19','batlq20','batlq21','batlq22','batlq23','batlq24','batlq25','batlq26','batlq27','batlq28','batlq29','batlq30','batlq31','batlq32','batlq33','batlq34','batlq35','batlq36','batlq37','batlq38','batlq39','batl_total','batlq_score12','batlq_score15','batlq_score21','batlq_score22','batlq_score32','batlq_score34','batlq_score35','batlq_score36','batlq_score37','batlq_score38']
b=b1+b2
cleanupfuncs.integercleanup(structure='batbil01',filePath="./prepped/hca/",varlist=b)
b2=['ed1_blood','bld_core','bld_core_grn','biospc_8','fasting','bld_core_snack','bld_core_spin','bld_rucdr','biospc_6','vitdlev','ls_alt','ls_ast','chloride','glucose','sodium','ed1_saliva','bld_core_d2ph','bld_core_d2pm','bld_core_p2fh','bld_core_p2fm','laba6']
cleanupfuncs.integercleanup(structure='bsc01',filePath="./prepped/hca/",varlist=b2)
d=['ddisc_valid']
cleanupfuncs.integercleanup(structure='deldisk01',filePath="./prepped/hca/",varlist=d)
#dd=['caffeine_s1yn','nicotine_s1yn','drug1_1','drug1_2','drug1_3','drug1_4','drug1_5','drug1_6','caffeine_s1preday','nicotine_s1preday']
#integercleanup(structure='drugscr01',filePath="./prepped/hca/",varlist=dd)
d2=['er40_c_cr','er40_c_ang','er40_c_fear','er40_c_hap','er40_c_noe','er40_c_sad','er40_c_fpa','er40_c_fpf','er40_c_fph','er40_c_fpn','er40_c_fps','er40_d_fc','er40_d_mc']
cleanupfuncs.integercleanup(structure='er4001',filePath="./prepped/hca/",varlist=d2)

In [ ]:
g=['gales1','gales2','gales3','gales4','gales5','gales6','gales6b','gales6c','gales7','gales8','gales9','gales10','gales11','gales12','gales13','gales14','gales15','gales16','gales17','gales18','gales19','gales20','gales21','gales21b','gales21c','gales22','gales23','gales24','gales25','gales26','gales_worst','gales1b','gales1c','gales3b','gales3c','gales10b','gales10c','gales13b','gales13c','gales16b','gales16c','gales18b','gales18c','gales19b','gales19c','gales20b','gales20c','gales2b','gales2c','gales7b','gales7c','gales9b','gales9c','gales25b','gales25c','gales22b','gales22c','gales23b','gales23c','gales24b','gales24c','gales26b','gales26c','gales8b','gales8c','gales11b','gales11c','gales12b','gales12c','gales4b','gales4c','gales17b','gales17c','gales5b','gales5c','gales14b','gales14c','gales15b','gales15c']
cleanupfuncs.integercleanup(structure='gales01',filePath="./prepped/hca/",varlist=g)

In [ ]:
i=['ipaq2','ipaq4','ipaq6','ipaq5a','ipaq5b','ipaq7a','ipaq7b','ipaq26n','ipaq3a','ipaq3b']
cleanupfuncs.integercleanup(structure='ipaq01',filePath="./prepped/hca/",varlist=i)

In [ ]:
l=['adlnow8','adlnow12','adlnow4','adlnow1','iadl05','iadl6','adlnow14','adlnow2','lbadl_currtot']
cleanupfuncs.integercleanup(structure='lbadl01',filePath="./prepped/hca/",varlist=l)

In [ ]:
la=['lan_know','lan611','lan612','lan613','lan621','lan622','lan623','lan71','lan72','lan81','lan82','lan631','lan632','lan633','lan73','lan83','lan641','lan642','lan643','lan74','lan84']
cleanupfuncs.integercleanup(structure='leap01',filePath="./prepped/hca/",varlist=la)
m=['scq_24_a','mchq_17','mchq_21','straw1','straw1a','straw1b','straw2','straw3','straw4a','straw4b','straw3a','straw2a','straw2b','straw3b']
cleanupfuncs.integercleanup(structure='mchq01',filePath="./prepped/hca/",varlist=m)
m2=['dental1','dental2','dental3','dental4','dental5','dental6']
cleanupfuncs.integercleanup(structure='medh01',filePath="./prepped/hca/",varlist=m2)
m3=['othmed0a','num_meds']
cleanupfuncs.integercleanup(structure='mendt01',filePath="./prepped/hca/",varlist=m3)

m4=['moca_total','moca_edu','visuospatial_executive','naming','attention_digit','attention_letter','attention_subtraction','mull_el_rs2','language_letter','abstraction','delayed_recall','orientation','visuospatial_executive','naming','attention_digit','attention_letter','attention_subtraction','mull_el_rs2','language_letter','abstraction','delayed_recall','orientation','visuospatial_executive','naming','attention_digit','attention_letter','attention_subtraction','mull_el_rs2','language_letter','abstraction','delayed_recall','orientation',]

cleanupfuncs.integercleanup(structure='moca01',filePath="./prepped/hca/",varlist=m4)
n1=['nffi_2','nffi_5','nffi_6','nffi_7','nffi_8','nffi_10','nffi_14','nffi_19','nffi_20','nffi_22','nffi_23','nffi_28','nffi_29','nffi_32','nffi_33','nffi_34','nffi_35','nffi_36','nffi_37','nffi_38','nffi_39','nffi_40','nffi_41','nffi_42','nffi_43','nffi_44','nffi_45','nffi_46','nffi_47','nffi_48','nffi_49','nffi_50','nffi_51','nffi_52','nffi_53','nffi_54','nffi_55','nffi_56','nffi_57','nffi_58','nffi_59','nffi_60']
n2=['nffi_1','nffi_3','nffi_4','nffi_9','nffi_11','nffi_12','nffi_13','nffi_15','nffi_16','nffi_17','nffi_18','nffi_21','nffi_24','nffi_25','nffi_26','nffi_27','nffi_30','nffi_31']
n=n1+n2
cleanupfuncs.integercleanup(structure='nffi01',filePath="./prepped/hca/",varlist=n)
p1=['parent_sleep20','csq_psqi_4_score','insomnia8']#,'psqip6b_5'
p2=['psqi5j_2','psqip6a_1','psqip6a_2','psqip6a_3','psqip6a_4','psqip6a_5','psqip6b_1','psqip6b_2','psqip6b_3','psqip6b_4','psqi5jalt','psqpi8','psqpi7_1','psqpi7_2','psqi_9','csq_psqi_2_score','csq_psqi_5a','psqi_component1','psqi_component2','psqi_component6','psqi_com2_3','psqi_com5','psqi_com7']
p=p1+p2
cleanupfuncs.integercleanup(structure='psqi01',filePath="./prepped/hca/",varlist=p)
r=['pea_ravlt_sd_tc','pea_ravlt_sd_trial_i_tc','pea_ravlt_sd_trial_ii_tc','pea_ravlt_sd_trial_iii_tc','pea_ravlt_sd_trial_iv_tc','pea_ravlt_sd_trial_v_tc','pea_ravlt_sd_listb_tc','pea_ravlt_sd_trial_vi_tc','delay_recall_intrusion','delay_total_intrusion','delay_total_repetitions']
cleanupfuncs.integercleanup(structure='ravlt01',filePath="./prepped/hca/",varlist=r)
s=['satisfaction1','satisfaction2','satisfaction3','satisfaction4','satisfaction7','satisfaction8','satisfaction9','satisfaction10','satisfaction11']
cleanupfuncs.integercleanup(structure='scan_debrief01',filePath="./prepped/hca/",varlist=s)